# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera
##Table of Contents
* [Introduction](#cd_introduction)
* [Data](#cd_data)


## Introduction <a name="cd_introduction"></a>
In this project, we will try to find an optimal location for a grocery store. Specifically, we'll be looking in the capital region of New York State and the surrounding area. 

Since there are plenty of grocery stores in the capital region, we are seeking to identify areas that

    a.) do not have a grocery store nearby, and
    b.) have a sufficient population density to support a grocery store. 


## Data <a name='cd_data'></a>
Based on the above description, the factors that influence our decision are:
    a.) The number of grocery stores in the area.
    b.) The distance to each grocery store.

Rather than using administrative boundaries (zipcode, county, city), we'll be generating the areas algorithmically. 
   
Starting with Albany, NY and radiating outward we will generate equally spaced hexagons until we have covered Albany, Schenectady and Troy as well as some of the surrounding area. 

For each of those areas, we'll be calculating the number of businesses in general, and the number of grocery stores specifically. We'll be looking for areas where the number of grocery stores and the number of businesses diverge sharply. 


In [ ]:
#!conda install pyproj --y
#!conda install shapely --y
#!conda install geopy --y
#!conda install folium --y
#!conda install pyproj --y
#!conda install shapely --y

from pprint import pprint
import json
import math
import geopy
import os
import configparser
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import FloatImage
import pyproj
import shapely.geometry
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import *
from sklearn.metrics import *
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm
%matplotlib inline
tqdm.pandas()

In [ ]:
# Create a geocoder object with an arbitrary user agent

geocoder = geopy.geocoders.Nominatim(user_agent='Capital_District_Notebook') 

# define a function to get geocoding from zip codes

def get_lat_long_locale(zip_code, geocoder=geocoder):
    """
    Takes a five character string representing the zip code, and returns 
    tuple representing the latitude and longitude of the zipcode center. 
    """
    location = geocoder.geocode({'postalcode':zip_code, "country":"us"})
    
    if location:
        return (location.latitude, location.longitude, location.address)
    else:
        return None




# Initial Neighborhood Candidatates
To get an initial sense of the geography, I'm using a list of zipcodes generated through a [tool provided by the US postal Service.]( 
https://www.unitedstateszipcodes.org/zip-code-radius-map.php)

In [ ]:
zipcodes = [
    12007, 12008, 12009, 12010, 12018, 12019, 12020, 12023, 12024, 12027,
    12033, 12040, 12041, 12042, 12045, 12046, 12047, 12052, 12053, 12054,
    12056, 12059, 12061, 12062, 12063, 12065, 12066, 12067, 12074, 12077,
    12083, 12084, 12085, 12086, 12087, 12094, 12106, 12110, 12115, 12118,
    12120, 12121, 12123, 12124, 12130, 12132, 12136, 12137, 12138, 12140,
    12143, 12144, 12147, 12148, 12150, 12151, 12153, 12154, 12156, 12157,
    12158, 12159, 12161, 12169, 12170, 12173, 12176, 12180, 12182, 12183,
    12184, 12185, 12186, 12188, 12189, 12192, 12193, 12195, 12196, 12198,
    12202, 12203, 12204, 12205, 12206, 12207, 12208, 12209, 12210, 12211,
    12222, 12302, 12303, 12304, 12305, 12306, 12307, 12308, 12309, 12863, 
    12866
] 
def zip_format(zip:int) -> str :
    """
    Takes zip codes as integers and returns 5 character strings.
    """
    return f"{zip:05d}"

zipcodes = list(map(zip_format, zipcodes)) # convert to strings

zipcodes = pd.DataFrame(data=zipcodes, columns=['zipcodes'])
zipcodes.head()

Now that we have our zipcodes and our geocoder, we're going to use the geocoder to pull the center of each zipcode. 

Since this involves querying an API, we want to minimize the number of requests that we're making. The following block of code attempts to load the data from a pickle file first. If the pickle file doesn't exist, it will query the API and create the pickle file. Therefore, if we run this block repeatedly it will only execute the query once. Afterwards it will load the data from the pickle file. 



In [ ]:

file = 'zipcode_location.pickle'
try:
    # Try to read from a pickle cache
    zipcodes = pd.read_pickle(file)
    # TQDM is a library for outputing feedback on a running process.
    # In this case, it will tell us that it's working from a locally
    # cached file. 
    tqdm.write(f'Working from file, {file}')
except:
    # Here we use TQDM to show that we aren't able to load a cache,
    # so.. we'll have to execute a query.
    tqdm.write('File not found. Querying...') 
    
    # tqdm modifies adds a method to the pandas DataFrame object called 
    # progress apply. Progress apply is like the apply function, 
    # but it includes a progress bar. 
    zipcodes['location'] = zipcodes['zipcodes'].progress_apply(
        get_lat_long_locale)
    
    # Lastly, we write the file...
    zipcodes.to_pickle(file)
    
    # ..and report out to the user.
    tqdm.write(f'Writing file, {file}')

zipcodes.head()


Our data is currently represented as zipcodes and location tuples. We want to convert our location tuples into latitude and longitude. To do that, we'll have to extract the latitude and longitude from the location column. 

We're going to do that with some simple throwaway functions (lambdas). 

In [ ]:
zipcodes['latitude'] = zipcodes['location'].apply(lambda x: x[0])
zipcodes['longitude'] = zipcodes['location'].apply(lambda x: x[1])


# The first item is always the city or town
zipcodes['city_or_town'] = zipcodes['location'].apply(
    lambda x: x[2].split(',')[0].strip())

# Extracting the remaining metadata is less straightforward. 
# Splitting the string and using negative indexing may look 
# a little strange, but it's the best way to get the correct values.

# get the county
zipcodes['county'] = zipcodes['location'].apply(
     lambda x: x[2].split(',')[-4].strip())

# Cleanup the value by chopping off the word county
zipcodes['county'] = zipcodes['county'].apply(lambda x: x.split(' ')[:-1][0])

# get the state
zipcodes['state'] = zipcodes['location'].apply(
     lambda x: x[2].split(',')[-3].strip())

# cleanup the value by using the two character acronym
zipcodes['state'].replace('New York', 'NY', inplace=True)

# drop the original location column
zipcodes.drop(columns="location", inplace=True)
zipcodes.head()

Once we've done that, we'll plot them on a map to get a better sense of **the area we intend to examine**.  This isn't the methodology that we outlined at the start. We're just trying to get a sense of the geography that we're working with. 


In the output that follows, red dots mark the zipcode center points. The green and purple circles mark is the approximate center of our area. The mean, and the median look to be virtually identical, meaning either variable should be useful for identifying our starting point. 


In [ ]:
try:
    # delete the map if it already exists (useful when running/editing cells)
    del capital_district_map
except:
    pass
center_mean = (
    zipcodes['longitude'].mean(), # approximate center of all our zipcodes
    zipcodes['latitude'].mean(),
)
center_median = (
    zipcodes['longitude'].median(), # approximate center of all our zipcodes
    zipcodes['latitude'].median(),
)

capital_district_map = folium.Map(center_mean[::-1], zoom_start=9)  
for ix, zipcode, lat, long, city, county, state in zipcodes.itertuples():
    label = f'{zipcode}, {city}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, long],
                        radius=2,
                        popup=label,
                        color='red',
                        opacity=.5,
                        fill=True,
                        fill_color='orange',
                        fill_opacity=0.1,
                        parse_html=False).add_to(capital_district_map)

folium.CircleMarker(center_mean[::-1],
                    radius=20,
                    popup='center',
                    color='green',
                    opacity=.5,
                    fill=True,
                    fill_color='green',
                    fill_opacity=0.1,
                    parse_html=False).add_to(capital_district_map)
folium.CircleMarker(center_median[::-1],
                    radius=20,
                    popup='center',
                    color='purple',
                    opacity=.5,
                    fill=True,
                    fill_color='purple',
                    fill_opacity=0.1,
                    parse_html=False).add_to(capital_district_map)

capital_district_map

As I stated before, the zip code data is just helpful for getting a frame of reference on our geography. We aren't going to rely on it for our final analysis. 

The reason for that decision is that the boundaries of zipcodes are fairly irregular. We want to be more fine grained. 

What we do next is create something like a hexagonal grid. We're using hexagons because they are compact. In a rectangular grid, the center point of each square would not be equidistant from each surrounding square. In a hexagonal grid, the centerpoint of each grid is equidistant from it's neighbors. 

To accurately calculate distances, however, we'll need to convert our data back and forth between long/lat coordinates and cartesian coordinates (in meters). The technique I'm using is demonstrated Data Scientist, Lyan Fu Fly. While his analysis focuses on the city of Berlin, this analysis focusses on an area of upstate NY. 

If you would like to read their analysis, you can find their medium article here: https://medium.com/@lyan.fu.fly/ibm-capstone-project-the-battle-of-neighborhoods-in-berlin-restaurants-dd326f1bfacb
And their github repository here: https://github.com/lingyan-f/ly.fu-Coursera-IBM-Cap-final-battle-of-the-neighborhoods-of-berlin/blob/master/Final%20Capstone%20Project-ly.fu.pdf


# Map Projection Functions


In [ ]:
def lonlat_to_xy(lon:float, lat:float) -> float:
    """
    This function takes the longitude and latitude of a goegraphic point, 
    and converts it to x,y coordinates on a cartestian grid. In order to
    convert the curved surface to a "flat" mapping, we need to select a 
    subsection of the globe and describe it as a rectangle. The pyproj library
    helps us to accomplish this. 
    """
    # define the source and destination projects
    src_proj = pyproj.Proj(proj='latlong', datum='WGS84')
    dst_proj = pyproj.Proj(proj="utm", zone=18, datum='WGS84')

    # Create a transformer object to transform the coordinates
    transformer = pyproj.transformer.Transformer.from_proj(src_proj, dst_proj)

    # Transform the coordinates
    x, y = transformer.transform(lon, lat)
    
    return x, y




def xy_to_lonlat(x:float, y:float) -> float:
    """
    This function uses a cartesian representation of the globe to 
    convert back to latitude and longitude. It's a limited use function, 
    as it only works for zone 18... but for this project, we only need 
    zone 18. If we wanted to extend this project to encompass new 
    locales, or to repeat the analysis globally, we  would need to build a more
    robust solution. 
    """
    # define the source and destination projects

    src_proj= pyproj.Proj(proj="utm", zone=18, datum='WGS84')
    dst_proj = pyproj.Proj(proj='latlong', datum='WGS84')

    # Create a transformer object to transform the coordinates
    transformer = pyproj.transformer.Transformer.from_proj(src_proj, dst_proj)

    # Transform the coordinates
    lon, lat = transformer.transform(x, y)
    
    return lon, lat



def calc_xy_distance(x1:float, y1:float, x2:float, y2:float) -> float:
    """
    Given two points, return the euclidean distance.
    """
    diff_x = x2 - x1
    diff_y = y2 - y1
    return np.sqrt(diff_x ** 2 + diff_y **2)

"""
Testing that our functions are working as intended. If we get can convert from
latitude and longitude and back again (getting our original values) then we are 
good shape. 
"""
print('Coordinate transformation check')
print('-------------------------------')
print('Capital District longitude={}, latitude={}'.format(*center_mean))
x, y = lonlat_to_xy(*center_mean)
print('Capital District UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Capital District  longitude={}, latitude={}'.format(lo, la))

It looks like our coordinate functions are working. We can convert to and from longitude and latitude and we get back the orignal values. 

Our next step is to create a grid of points that are related hexagonally.  

To create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**. Another way of discribing this is to say that each point has six neighboring points and that each neighboring point is located on a thirty degree line. 

In [ ]:
try:
    del latitudes
    del longitudes
except:
    pass


def generate_hex_grid(center=center_mean):
    # first define our city center in Cartesian coordinates
    center_x, center_y = lonlat_to_xy(*center)  

    
    # k is the coef of a 30 degree angle on the unit circle
    
    k = math.sqrt(3) / 2 
    
    # get our minimum x value. In this case, we're starting  
    # 50 kilometers west of the center point
    x_min = center_x - 50000 
    
    x_step = 2500 # two and a half kilometers increments
    
    # because the hexagons "settle into" the crevices of the row beneath them, 
    # we can fit more hexagons on the vertical axis than the horizontal axis
    
    y_min = center_y - 50000 - (int(126 / k) * k * 2500 - 100000) / 2
    y_step = 2500 * k # 2 and a half kilometers * sqrt(3)/2

    latitudes = []
    longitudes = []
    distances_from_center = []
    xs = []
    ys = []
    # iterate over y rows as [i]
    for i in range(0, int(126 / k)):
        
        # First we identify our vertical incrementation to locate 
        # our row.
        # In this case y is equal to the minimum plus an incrementation
        # based on the y_step value. 
        y = y_min + i * y_step
        
        # the row is shifted right by a half hexagon when the row is even
        x_offset = 1250 if i % 2 == 0 else 0
        
        for j in range(0, 126):
            # Now that we have identified the center of our row
            # we derive a simpler calculation for the east/west
            # positioning
            x = x_min + j * x_step + x_offset
            distance_from_center = calc_xy_distance(center_x, center_y, x, y)
            # this step  is a bit of a brute force method. 
            # we're cutting off any points that are more than 50.05 kilometers
            # from the center of our points. This effectively turns our grid into 
            # a circle. 
            if (distance_from_center <= 50050):
                lon, lat = xy_to_lonlat(x, y)
                latitudes.append(lat)
                longitudes.append(lon)
                distances_from_center.append(distance_from_center)
                xs.append(x)
                ys.append(y)
    # lastly we construct a dataframe
    neighborhoods = pd.DataFrame({
        'latitude': latitudes,
        'longitude': longitudes,
        'x': xs,
        'y': ys,
        'distance': distances_from_center
    })
    # and return it
    return neighborhoods


neighborhoods = generate_hex_grid(center_mean)
neighborhoods.to_csv('neighborhoods.csv')

neighborhoods.shape

This next step shows the results of our work. We use the hexagonal positioning to place densely packed circles onto our map. 

The geography includes rural, suburban and urban regions. It should be helpful for us. 

In [ ]:
hex_map = folium.Map(location=center_mean[::-1], zoom_start=9)

folium.Circle(center_mean[::-1], radius=50050).add_to(hex_map)

# Iterate across the center of each "hex" and draw a circle around it. The radius of the circle is 1.25k. 
# If you recall from the prior functions, our neighborhoods are spaced with a diameter of 2.5k. 
for _, lat, long in neighborhoods[['latitude', 'longitude']].itertuples():
    label = f'{lat, long}'
    label = folium.Popup(label, parse_html=True)
    folium.Circle([lat, long],
                  radius=1250,
                  popup=label,
                  color='red',
                  opacity=.1,
                  fill=True,
                  fill_color='red',
                  fill_opacity=0.01,
                  parse_html=False).add_to(hex_map)

hex_map

Here's a quick glimpse at what our coordinate system looks like

In [ ]:
neighborhoods.head()

# The real data
Finally. We're getting somewhere. In the next few steps we're getting ready 
to query the foursquare api. 

Unfortunately, this part requires some security precautions. I have my credentials stored in a .env 
file which probably won't be available to you. If you would like to run this on your own, you can 
setup a developer account and create your own version of the file. 

The file should be named ".env" and stored in the same directory as this notebook. After you 
create the file paste in the following:
```
[foursquare]
client_id=yourKeyHere
client_secret=yourSecretHere
 ```
Then, replace the relevant parts with your own credentials. If you need to register for a developer account, you can do that [here](https://foursquare.com/developers/signup).


In [ ]:
# Getting the foursquare API credentials from a 
# .env file.
config = configparser.ConfigParser()
config.read_file(open('.env'))

# The url of the foursquare api
url = 'https://api.foursquare.com/v2/venues/explore'

# Loading our secrets from a .env file

fsq_params = {
    'client_id': config.get('foursquare', 'client_id'),
    'client_secret': config.get('foursquare', 'client_secret'),
    #'v': '20200222'
}


def query_fsquare_for_nearby_venues(lat,
                                    long,
                                    limit=2000,
                                    radius=20000,
                                    credentials=fsq_params,
                                    check_cache=False):
    """
    Query the foursquare API using latitude and longitude, return venues that 
    are withing a given radius of the point. This function first checks for a 
    locally available csv (which indicates that the function has already been run).
    If no such csv is available, it executes the query. 
    """
    if check_cache:
        cache = pd.read_csv(check_cache,
                            usecols=['center_lat',
                                     'center_long']).drop_duplicates()
        in_cache = cache[(cache['center_lat'] == lat)
                         & (cache['center_long'] == long)]

        if in_cache.empty:
            credentials['ll'] = '{},{}'.format(lat, long),
            credentials['limit'] = limit,
            credentials['radius'] = radius
            resp = requests.get(url=url, params=credentials)
            return {'lat': lat, 'long': long, 'resp': resp}
    else:
        credentials['ll'] = '{},{}'.format(lat, long),
        credentials['limit'] = limit,
        credentials['radius'] = radius
        resp = requests.get(url=url, params=credentials)
        return {'lat': lat, 'long': long, 'resp': resp}


def normalize_response(response):
    """
    A helper function to clean up the response from the 
    query_fsquare_for_nearby_venues function.
    
    This extracts the results from json and processes them into a dataframe. 
    """
    results = response['resp'].json()
    results = pd.json_normalize(results['response']['groups'][0]['items'])
    try:
        results = results[[
            'venue.name', 'venue.location.lat', 'venue.location.lng',
            "venue.categories"
        ]]
        results.iloc[:, -1] = results['venue.categories'].map(
            lambda x: x[0]['shortName'])
    except KeyError as e:
        #mock response venue
        results = pd.DataFrame(
            {
                'venue.name': 'None',
                'venue.location.lat': response['lat'],
                'venue.location.lng': response['long'],
                'venue.categories': 'None'
            },
            index=[0])
    results['center_lat'] = response['lat']
    results['center_long'] = response['long']
    results.columns = [
        'venue', 'latitude', 'longitude', 'category', 'center_lat',
        'center_long'
    ]
    return results


def cache_successful_response(successful_response_data):
    """
    Is this necessary?
    """
    try:  #look for existing cache
        previously_cached_data = pd.read_csv('response200.csv', index_col=0)
        new_records = len(successful_response_data)
        old_records = len(previously_cached_data)
        results = pd.concat([successful_response_data, previously_cached_data])
        results.drop_duplicates(inplace=True)
        non_duplicates = len(results) - old_records
        results.to_csv('response200.csv')
        return old_records, new_records, non_duplicates

    except FileNotFoundError as e:
        tqdm.write(str(e))
        successful_response_data.to_csv('response200.csv')
        return 0, 0, 0

In [ ]:
fails = []
check_cache = False

for ix, lat, long in tqdm(
        list(neighborhoods[['latitude', 'longitude']].itertuples())):
    q = query_fsquare_for_nearby_venues(lat, long, check_cache=check_cache)
    
    if q is None:
        # If we don't get a valid response from foursquer
        next
    elif q['resp'].status_code == 200:
        #if we do get a valid response, normalize it.Then check for duplicates. Then 
        success = normalize_response(q)
        old, new, nondupe = cache_successful_response(success)
        #tqdm.write('{} records in cache. Comparing {}. {} new records'.format(old, new, nondupe))
    elif q['resp']:
        tqdm.write('{} {} {}'.format(q['lat'], q['long'],
                                     q['resp'].status_code))
        fails.append((lat, long, q['resp'].status_code))
    check_cache = "response200.csv"

In [ ]:
# checking file size and whether any query point failed
fsq = pd.read_csv('response200.csv', index_col=0)
print(f"""
shape of dataset: {fsq.shape}
shape of points dataset {neighborhoods.shape}
number of unique centers in new dataset:{fsq["center_lat"].nunique(), fsq["center_long"].nunique()}
""")

In [ ]:
#checking head
fsq.head()

In [ ]:
fsq.groupby(["center_lat", "center_long"]).count().sort_values("venue")

In [ ]:
fsq[fsq.category.str.contains("Grocery")].sample(5)

In [ ]:
fsq.category.nunique()

There are too many categories to get our clustering algorithm going, and doing text analysis is a little beyond the scope of this project.

I've opted to create a mapping file that categorizes venues into a few groups:

* Convenience (Corner stores, bodegas, delis, etc.)
* Food (Restauarants, or convenience food)
* Grocery (Grocery stores & supermarkets)
* Outdoors (Parks and natural features, or poi that indicate access to a natural feature)
* Entertainment (Movies, Theatres, Museums, Stadiums and Nightlife(other than food)
* Retail (Most stores)
* Commercial (Services, or business to business stores)
* Other (Things that didn't fit well)


There's a bit of rough justice in selecting these categories. We're going to see how they work out, and if they work poorly we'll revisit them. 

In [ ]:
# Open our mapping file and create a dictionary to reduce the categories
with open("mapping") as f:
    
    mapping ={item[0]:item[1] for item in [ln.strip().split(':') for ln in f]}
    
    # Café isn't working right, probably beccause of the accented character. 
    # Something to look at more closely when I have the time. 
    # For now, just fix it. 
    mapping["Café"] = "Food"


fsq["sec_category"] = fsq["category"]
fsq["category"] = fsq.category.replace(mapping)


fsq = fsq[fsq["category"].isin((fsq.category.value_counts() > 3).index)]

In [ ]:
category_counts = fsq.category.value_counts()
category_counts = category_counts[category_counts>3]
fsq = fsq[fsq["category"].isin(category_counts.index)]
fsq.category.value_counts()

Next we're going to plot our venues out to our map. If we do it well, we should start to see urban areas show up darker. 

In [ ]:
try:
    del capital_district_locations
except:
    pass
capital_district_locations = folium.Map(center_mean[::-1], zoom_start=9)
for _, venue, lat, long, category, clat, clong, sec_category in tqdm(list(fsq.itertuples())):
    label = f'{category}, {sec_category}'
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker([lat, long],
                        radius=2,
                        popup=label,
                        color='blue',
                        opacity=.15,
                        fill=True,
                        fill_color='lightblue',
                        fill_opacity=0.1,
                        parse_html=False).add_to(capital_district_locations)

capital_district_locations

That was pretty succcessful. 

In [ ]:
colors = {
"Food":"red",
"Retail":"orange",
"Commercial":"blue",
"Outdoors":"green",
"Entertainment":"yellow",
"Convenience":"white",
"Grocery":"cyan",
"Other":"purple",
"Rural":"brown",
"Gas/Highway/Reststop":"black"}

try:
    del capital_district_locations
except:
    pass
capital_district_locations = folium.Map(center_mean[::-1], zoom_start=9)
for _, venue, lat, long, category, clat, clong, sec_category in tqdm(list(fsq.itertuples())):
    label = f'{category}, {sec_category}'
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker([lat, long],
                        radius=2,
                        popup=label,
                        color=colors[category],
                        opacity=.15,
                        fill=True,
                        fill_color=colors[category],
                        fill_opacity=0.1,
                        parse_html=False).add_to(capital_district_locations)

capital_district_locations

In [ ]:
colors = {
"Food":"red",
"Retail":"blue",
"Commercial":"blue",
"Outdoors":"green",
"Entertainment":"blue",
"Convenience":"white",
"Grocery":"cyan",
"Other":"purple",
"Rural":"purple",
"Gas/Highway/Reststop":"black"}

try:
    del capital_district_locations
except:
    pass
capital_district_locations = folium.Map(center_mean[::-1], zoom_start=9)
for _, venue, lat, long, category, clat, clong, sec_category in tqdm(list(fsq.itertuples())):
    label = f'{category}, {sec_category}'
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker([lat, long],
                        radius=2,
                        popup=label,
                        color=colors[category],
                        opacity=.15,
                        fill=True,
                        fill_color=colors[category],
                        fill_opacity=0.1,
                        parse_html=False).add_to(capital_district_locations)

capital_district_locations

In [ ]:
try:
    del grocery_map
except:
    pass
grocery_map = folium.Map(center_mean[::-1], zoom_start=9)

for _, venue, lat, long, category, clat, clong, sec_category in tqdm(list(fsq.itertuples())):
    label = f'{category}, {sec_category}'
    label = folium.Popup(label, parse_html=True)
    
    if category != 'Grocery':
        folium.CircleMarker([lat, long],
                            radius=3,
                            popup=label,
                            color='purple',
                            opacity=.025,
                            fill=True,
                            fill_color='purple',
                            fill_opacity=0.1,
                            parse_html=False).add_to(grocery_map)
    else:
        
        folium.CircleMarker([lat, long],
                            radius=10,
                            popup=label,
                            color='green',
                            opacity=.5,
                            fill=True,
                            fill_color='green',
                            fill_opacity=0.1,
                            parse_html=False).add_to(grocery_map)

grocery_map

# First attempt at modeling

To generate our features we're going to create dummies from our primary categories and then merge them back onto the original dataframe. 

In [ ]:
fsq = pd.concat([fsq, pd.get_dummies(fsq.category)], axis=1)

fsq = fsq.loc[:,~fsq.columns.duplicated()] # If the cell is accidentally run more than once, this will drop the duplicate columns. 
fsq["poi"] = pd.get_dummies(fsq.category).sum(axis=1)
fsq

In [ ]:
agg=     fsq.groupby(['center_lat', 'center_long'])["Commercial", "Convenience", "Entertainment", 
                                                    "Food", "Gas/Highway/Reststop", "Grocery", 
                                                    "Other", "Outdoors", "Retail", 
                                                    "Rural", "poi"].agg(["mean", "sum"])
agg.columns = agg.columns.to_flat_index()
agg.columns = [f"{col[0]}_{col[1]}" for col in agg.columns]
agg

## Summarize, and scale

In [ ]:
scaled = agg.copy()

scaled.iloc[:,:] = StandardScaler().fit_transform(X=agg.iloc[:, :])


In [ ]:
scaled.head()

In [ ]:

ks = list(range(1, 15))
sse = []
for k in ks:
    kmm = KMeans(n_clusters=k)
    kmm.fit(scaled.iloc[:, :21])

    sse.append(kmm.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(ks, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance')
plt.show()

Our dataset doesn't seem to like the standard approach to kmeans. 

There's no clear elbow in our scree plot. We'll try and do a little visual inspection by collapsing our data into two dimensions, and evaluating the clusters visually. If we were to make a guess from this, we have at most 4 clusters. 

In [ ]:

pca_outp = pd.DataFrame(PCA(2).fit_transform(scaled.iloc[:, :21]))
pca_outp.columns = "X", "Y"
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(10,10) )
M = KMeans
for k, ax in zip(range(2, 8), axs.flatten(), ):
    m=M(k)
    m.fit(scaled.iloc[:,:21])
    ypred= m.fit_predict(scaled.iloc[:, :21])
    sc = silhouette_score(scaled.iloc[:,:21],ypred, )
    sns.scatterplot(x="X", y="Y", data=pca_outp, c=ypred, ax=ax)
    ax.set_title(f"k={k}, silhoute:{sc}")
plt.tight_layout()

In [ ]:
X = scaled.iloc[features]
pca_outp = pd.DataFrame(PCA(2).fit_transform(X))
pca_outp.columns = "X", "Y"
trials =16
fig, axs = plt.subplots(nrows=trials//2, ncols=2, figsize=(10,trials*3) )
for eps, ax in zip(np.linspace(2.2,2.5,trials), axs.flatten(), ):
    m=DBSCAN(eps=eps, )
    ypred= m.fit_predict(scaled.iloc[:, :21])
    sc = silhouette_score(scaled.iloc[:,:21],ypred, )
    sns.scatterplot(x="X", y="Y", data=pca_outp, c=ypred, ax=ax)
    ax.set_title(f"eps={eps:.4f}, silhoute:{sc:.4f}")
plt.tight_layout()

In [ ]:
X = scaled.iloc[features]
pca_outp = pd.DataFrame(PCA(2).fit_transform(X))
pca_outp.columns = "X", "Y"
from itertools import product

trials = product([
fig, axs = plt.subplots(nrows=trials//2, ncols=2, figsize=(10,trials*3) )

for eps, ax in zip(np.linspace(2.2,2.5,trials), axs.flatten(), ):
    m=Birch()
    ypred= m.fit_predict(scaled.iloc[:, :21])
    sc = silhouette_score(scaled.iloc[:,:21],ypred, )
    sns.scatterplot(x="X", y="Y", data=pca_outp, c=ypred, ax=ax)
    ax.set_title(f"eps={eps:.4f}, silhoute:{sc:.4f}")
plt.tight_layout()

In [ ]:
scaled = scaled.iloc[:,:21]

# Evaluate Model


## Visualizing distinguishing features between clusters

In [ ]:
x_scale = agg.max().max()  # for setting a common x-axis scale

In [ ]:
clusters = list(range(agg.KMeans_5.nunique()))

cluster_box_plots = {n: n for n in clusters}
for cluster in clusters:
    cluster_summary = agg[agg['KMeans_5'] == cluster]
    cluster_summary = cluster_summary.drop(columns=['KMeans_5'])
    top5_filter = cluster_summary.mean().sort_values(ascending=False)[:5].index
    cluster_top5 = cluster_summary[top5_filter]

    title = "Cluster_{}".format(cluster)
    plt.figure(figsize=(4, 2.5))
    plt.tight_layout()
    cluster_box_plots[cluster] = sns.boxplot(data=cluster_top5,
                                             orient='h',
                                             palette="Set2")
    cluster_box_plots[cluster].set_title(title)
    cluster_box_plots[cluster].set_xbound(0, x_scale)
    cluster_box_plots[cluster].set_xticks([])
    cluster_box_plots[cluster].figure.savefig('{}.png'.format(title),
                                              bbox_inches='tight')

## Visualizing Clusters Geographically

### All Clusters

In [ ]:
try:
    del results_map
except:
    pass
colors = {
    0: 'red',
    1: 'green',
    2: 'navy',
    3: 'orange',
    4: 'violet',
    5: "yellow",
}

results_map = folium.Map(location=center[::-1], zoom_start=9)

folium.Circle(center[::-1], radius=50050).add_to(results_map)

for latlong, cluster in scored_profile['cluster'].iteritems():
    label = f'{latlong}'
    label = folium.Popup(label, parse_html=True)
    folium.Circle(latlong,
                  radius=1200 * 1.2,
                  popup=label,
                  color=colors[cluster],
                  opacity=.1,
                  fill=True,
                  fill_color=colors[cluster],
                  fill_opacity=0.25,
                  parse_html=False).add_to(results_map)
results_map

In [ ]:
offset_center = center[0] + .2, center[1] + .05,

### Cluster 0

In [ ]:
try:
    del cluster0_map
except:
    pass

cluster0_map = folium.Map(location=offset_center[::-1], zoom_start=9)
cluster0_iterable = scored_profile[scored_profile['cluster'] == 0]
folium.Circle(center[::-1], radius=50050).add_to(cluster0_map)

for latlong in cluster0_iterable.index:
    label = f'{latlong}'
    label = folium.Popup(label, parse_html=True)
    folium.Circle(latlong,
                  radius=1200 * 1.2,
                  popup=label,
                  color=colors[cluster],
                  opacity=.1,
                  fill=True,
                  fill_color=colors[cluster],
                  fill_opacity=0.25,
                  parse_html=False).add_to(cluster0_map)

FloatImage("Cluster_0.png", bottom=70, left=62).add_to(cluster0_map)
cluster0_map

### Cluster 1

In [ ]:
try:
    del cluster1_map
except:
    pass

cluster1_map = folium.Map(location=offset_center[::-1], zoom_start=9)
cluster1_iterable = scored_profile[scored_profile['cluster'] == 1]

folium.Circle(center[::-1], radius=50050).add_to(cluster1_map)

for latlong in cluster1_iterable.index:
    label = f'{latlong}'
    label = folium.Popup(label, parse_html=True)
    folium.Circle(latlong,
                  radius=1200 * 1.2,
                  popup=label,
                  color=colors[1],
                  opacity=.1,
                  fill=True,
                  fill_color=colors[1],
                  fill_opacity=0.25,
                  parse_html=False).add_to(cluster1_map)

FloatImage("Cluster_1.png", bottom=70, left=62).add_to(cluster1_map)
cluster1_map

### Cluster 2

In [ ]:
try:
    del cluster2_map
except:
    pass

cluster2_map = folium.Map(location=offset_center[::-1], zoom_start=9)
cluster2_iterable = scored_profile[scored_profile['cluster'] == 2]

folium.Circle(center[::-1], radius=50050).add_to(cluster2_map)

for latlong in cluster2_iterable.index:
    label = f'{latlong}'
    label = folium.Popup(label, parse_html=True)
    folium.Circle(latlong,
                  radius=1200 * 1.2,
                  popup=label,
                  color=colors[2],
                  opacity=.1,
                  fill=True,
                  fill_color=colors[2],
                  fill_opacity=0.25,
                  parse_html=False).add_to(cluster2_map)

FloatImage("Cluster_2.png", bottom=70, left=62).add_to(cluster2_map)
cluster2_map

### Cluster 3

In [ ]:
try:
    del cluster3_map
except:
    pass

cluster3_map = folium.Map(location=offset_center[::-1], zoom_start=9)
cluster3_iterable = scored_profile[scored_profile['cluster'] == 3]

folium.Circle(center[::-1], radius=50050).add_to(cluster3_map)

for latlong in cluster3_iterable.index:
    label = f'{latlong}'
    label = folium.Popup(label, parse_html=True)
    folium.Circle(latlong,
                  radius=1200 * 1.2,
                  popup=label,
                  color=colors[3],
                  opacity=.1,
                  fill=True,
                  fill_color=colors[3],
                  fill_opacity=0.25,
                  parse_html=False).add_to(cluster3_map)

FloatImage("Cluster_3.png", bottom=70, left=62).add_to(cluster3_map)
cluster3_map

# Cluster 4

In [ ]:
try:
    del cluster4_map
except:
    pass

cluster4_map = folium.Map(location=offset_center[::-1], zoom_start=9)
cluster4_iterable = scored_profile[scored_profile['cluster'] == 4]

folium.Circle(center[::-1], radius=50050).add_to(cluster4_map)

for latlong in cluster4_iterable.index:
    label = f'{latlong}'
    label = folium.Popup(label, parse_html=True)
    folium.Circle(latlong,
                  radius=1200 * 1.2,
                  popup=label,
                  color=colors[4],
                  opacity=.1,
                  fill=True,
                  fill_color=colors[4],
                  fill_opacity=0.25,
                  parse_html=False).add_to(cluster4_map)

FloatImage("Cluster_4.png", bottom=70, left=62).add_to(cluster4_map)
cluster4_map

In [ ]:
# try:
#     del cluster5_map
# except:
#     pass

# cluster5_map = folium.Map(location=offset_center[::-1], zoom_start=9)
# cluster5_iterable = scored_profile[scored_profile['cluster']==5]

# folium.Circle(center[::-1], radius=50050).add_to(cluster5_map)

# for latlong in cluster5_iterable.index:
#     label = f'{latlong}'
#     label = folium.Popup(label, parse_html=True)
#     folium.Circle(latlong,
#                   radius=1200 * 1.2,
#                   popup=label,
#                   color=colors[5],
#                   opacity=.1,
#                   fill=True,
#                   fill_color=colors[5],
#                   fill_opacity=0.25,
#                   parse_html=False).add_to(cluster5_map)

# FloatImage("Cluster_5.png", bottom=70, left=62 ).add_to(cluster5_map)
# cluster5_map

In [ ]:
venues

In [ ]:
for a, b, lat, long, category, generic, in tqdm(list(venues.itertuples())):
    label = f'{generic}, {category}'
    label = folium.Popup(label, parse_html=True)
    if generic == 'Groceries':
        folium.Circle([lat, long], radius=625, color='Black',
                      fill=False).add_to(results_map)
results_map

In [ ]:
centers = neighborhoods[['center_lat', 'center_long']].copy().drop_duplicates()

xs, ys = [], []

for _, lat, long in centers.itertuples():
    x, y = lonlat_to_xy(long, lat)
    xs.append(x)
    ys.append(y)

In [ ]:
centers['x'] = xs
centers['y'] = ys

In [ ]:
def query_fsquare_for_grocery_stores(latitude,
                                     longitude,
                                     limit=100,
                                     radius=30000,
                                     query='groceries',
                                     credentials=fsq_params):

    credentials['ll'] = '{},{}'.format(lat, long),
    credentials['limit'] = limit,
    credentials['radius'] = radius
    credentials['query'] = query
    resp = requests.get(url=url, params=credentials)
    return {'lat': lat, 'long': long, 'resp': resp}

In [ ]:
def df_apply_latlong_to_xy(df):
    df = df[['latitude', "longitude"]]
    df.drop_duplicates(inplace=True)

    xs, ys = [], []

    for _, lat, long in df.itertuples():
        x, y = lonlat_to_xy(long, lat)
        xs.append(x)
        ys.append(y)

    df['x'] = xs
    df['y'] = ys
    return df

In [ ]:
groceries = neighborhoods[neighborhoods['gen_cat'] == 'Groceries']
first_pass = True

print(groceries.shape)
for neighborhood in outer_rim:
    resp = query_fsquare_for_grocery_stores(**outer_rim.iloc[1, :2].to_dict())
    resp = normalize_response(resp)
    resp = resp[(resp['category'] == 'Supermarket') |
                (resp['category'] == 'Grocery Store')]
    if first_pass:
        beyond_border_grocery_stores = resp
    else:
        beyond_border_grocery_stores = pd.concat(
            [beyond_border_grocery_stores, resp])
        first_pass = False

print(beyond_border_grocery_stores.shape)
beyond_border_grocery_stores = beyond_border_grocery_stores[[
    'latitude', 'longitude'
]]
beyond_border_grocery_stores = beyond_border_grocery_stores.drop_duplicates()
groceries = groceries[['latitude', "longitude"]].copy().drop_duplicates()

In [ ]:
beyond_border_grocery_stores = beyond_border_grocery_stores[[
    'latitude', 'longitude'
]]
beyond_border_grocery_stores = beyond_border_grocery_stores.drop_duplicates()
groceries = groceries[['latitude', "longitude"]].copy().drop_duplicates()

In [ ]:
groceries = pd.concat([beyond_border_grocery_stores,
                       groceries]).drop_duplicates()
groceries = df_apply_latlong_to_xy(groceries)

In [ ]:
nearest_grocer = []
for _, orig_x, orig_y in centers[['x', 'y']].itertuples():
    distances = []
    for _, dest_x, dest_y in groceries[['x', 'y']].itertuples():
        distance = calc_xy_distance(orig_x, orig_y, dest_x, dest_y)
        distances.append(distance)
    nearest_grocer.append(min(distances))
centers['dist_to_grocer'] = nearest_grocer
centers.set_index(['center_lat', "center_long"], inplace=True)

In [ ]:
centers.sort_index(inplace=True)
scored_profile.sort_index(inplace=True)

In [ ]:
scored_profile['Grocer_Dist'] = centers.distance_to_groceries

In [ ]:
def plot_hists(df, categorical, continuous):
    for n in sorted(scored_profile.cluster.value_counts().index):
        print(n)
        scored_profile[continuous].plot(kind='hist', bins=100, alpha=.3)
        scored_profile.loc[scored_profile[categorical] == n, continuous].plot(
            kind='hist', bins=100, alpha=.3)
        plt.title('{} {}'.format(categorical, n))
        plt.show()


plot_hists(scored_profile, 'cluster', 'Grocer_Dist')

In [ ]:
scored_profile['scaled_dist'] = None

for n in range(scored_profile.cluster.nunique()):
    print(n)
    clust = scored_profile.loc[scored_profile['cluster'] ==
                               n, ["Grocer_Dist", "scaled_dist"]]
    print(clust.shape)
    scaler = StandardScaler()
    clust['scaled_dist'] = scaler.fit_transform(
        clust['Grocer_Dist'].values.reshape(-1, 1))
    scored_profile.loc[scored_profile['cluster'] ==
                       n, 'scaled_dist'] = clust['scaled_dist']

In [ ]:
plot_hists(scored_profile, 'cluster', 'scaled_dist')

In [ ]:
first_pass = True
for c in scored_profile['cluster'].value_counts().index:
    clust = scored_profile.loc[scored_profile['cluster'] == c, "scaled_dist"]

    if first_pass:
        prospective_locations = clust.sort_values(ascending=False)[:3]

    else:
        prospective_locations = pd.concat(
            [prospective_locations,
             clust.sort_values(ascending=False)[:3]])
    first_pass = False

In [ ]:
try:
    del results_map
except:
    pass
colors = {
    0: 'red',
    1: 'green',
    2: 'navy',
    3: 'orange',
    4: 'violet',
    5: "yellow",
}

results_map = folium.Map(location=center[::-1], zoom_start=9)

folium.Circle(center[::-1], radius=50050).add_to(results_map)
folium.Circle(center[::-1], radius=80000).add_to(results_map)

for latlong, cluster in scored_profile['cluster'].iteritems():
    label = f'{latlong}'
    label = folium.Popup(label, parse_html=True)
    folium.Circle(latlong,
                  radius=1200 * 1.2,
                  popup=label,
                  color=colors[cluster],
                  opacity=.1,
                  fill=True,
                  fill_color=colors[cluster],
                  fill_opacity=0.25,
                  parse_html=False).add_to(results_map)
results_map

for _, lat, long, in tqdm(
        list(groceries[['latitude', 'longitude']].itertuples())):

    folium.Circle([lat, long], radius=625, color='Black',
                  fill=False).add_to(results_map)

for latlong, _ in tqdm(list(prospective_locations.iteritems())):

    folium.Circle(latlong, radius=625, color='Red',
                  fill=False).add_to(results_map)
results_map

In [ ]:
ethan = zipcodes[["zipcodes",'latitude', 'longitude']].loc[zipcodes['zipcodes']=="12065"]
ethan.iloc[0,0]

In [ ]:
folium.Marker(ethan.iloc[0,1:]).add_to(results_map)
results_map

In [ ]:
heat_data
H